In [9]:
# 1. Reverse words
import sys

def reverse_word(string):
    return "".join([string[i].lower() for i in range(len(string)-1, -1, -1) if string[i].isalnum()])

for line in sys.stdin:
    splited = line.split()
    for i in range(len(splited)):
        splited[i] = reverse_word(splited[i])
    splited = splited[::-1] # reverse the list of words
    first_word = splited[0]
    first_word = first_word[0].upper() + first_word[1:]
    splited[0] = first_word
    print(" ".join(splited))

In [ ]:
# Test using the command in windows
"type test.txt | python reverse.py"

In [47]:
# 2. Python sum of logP
import math

def sum_log(lst):
    # transform the string into floating numbers
    nums = [float(num) for num in lst]
    
    # find the maximum num
    logPMax = -sys.float_info.max
    for num in nums:
        if num > logPMax:
            logPMax = num
    
    # Caculate the logP using the formula
    # Let's use Pmax * di to represent Pi, then log(di) = log(Pi) - log(Pmax)
    # log(P1+P2+...Pn) = log(Pmax*d1 + Pmax*d2...+Pmax*dn) = logPmax + [log(d1)+log(d2)+...log(dn)]
    sum_delta = 0
    for i in range(len(nums)):
        log_delta = nums[i] - logPMax
        sum_delta += math.exp(log_delta)
    
    return round((logPMax + math.log(sum_delta)), 3)

In [48]:
# Test
import math
lst = [0.01, 0.09, 0.7,0.15, 0.05]
log_lst = [math.log(num) for num in lst]
print("Original Ps are {}".format(lst))
print("Input Log(P)s are {}".format(log_lst))
print("Return should be 0 since the original Ps sums to 1. Return:", sum_log(log_lst))

Original Ps are [0.01, 0.09, 0.7, 0.15, 0.05]
Input Log(P)s are [-4.605170185988091, -2.4079456086518722, -0.35667494393873245, -1.8971199848858813, -2.995732273553991]
('Return should be 0 since the original Ps sums to 1. Return:', -0.0)


In [3]:
# 3.Likelihood of Unseen Sentence
from collections import defaultdict
import math

def count_V(token_set):
    """
    Count training vocabulary size
    """
    V = len(token_set)
    
    if "." in token_set:
        V -= 1
    if " " in token_set:
        V -= 1
    if "," in token_set:
        V -= 1
    
    return V


def train_oneSent(lst,n,dic_n1,dic_n):
    """
    Train on one sentence list (w/o ending period), update the count of n-gram in dictionary
    and the count of (n-1)-gram in dictionary
    """
    if len(lst) < n-1:
        return # if there isn't enough length even to update (n-1) gram
    
    for i in range(0, len(lst)-(n-1)+1):
        n1_gram = tuple(lst[i:(i+n-1)])
        dic_n1[n1_gram] += 1 # default dict for (n-1) gram
        
        if i < len(lst)-n+1: # if there can be an ngram 
            n_gram = tuple(lst[i:(i+n)])
            dic_n[n_gram] += 1 # default dict for ngram

def smoothed_prob(ngram_lst,V,dic_n1,dic_n):
    """
    Given the count dictionaries and get the smoothed probability of wn,conditioned on wn-1...w1
    ngram_lst is splited
    """
    n1_gram = tuple(ngram_lst[:-1])
    n_gram = tuple(ngram_lst)
    
    numer = dic_n[n_gram] + 1
    denom = dic_n1[n1_gram] + V
    
    return numer*1.0/denom

def predict(sent_lst,n,V,dic_n1,dic_n):
    """
    Given a sentence, predict its likelihood
    """
    if len(sent_lst) < n:
        return round(0, 3)
    
    logP = 0
    for i in range(0, len(sent_lst)-n+1):
        n_gram = sent_lst[i:(i+n)]
        logP += math.log(smoothed_prob(n_gram, V, dic_n1, dic_n))
    
    P = math.exp(logP)
    return round(P, 3)

def train_andPred(string):
    """
    Main function, train the ngram model and predicts
    """
    all = string.split(",")
    n = int(all[0])
    train = all[1].split(".") # split training corp into sentences
    test_sent = all[2]
    if test_sent[-1] == ".":
        test_sent = test_sent[:-1]
    
    dic_n1 = defaultdict(int)
    dic_n = defaultdict(int)
    
    # Train the model
    tokens= set([])
    for sent in train:
        train_oneSent(sent.split(), n, dic_n1, dic_n)
        for token in sent.split():
            tokens.add(token)
    V = count_V(tokens)
    
    # Predict the test sentence
    return predict(test_sent.split(), n, V, dic_n1, dic_n)

In [4]:
# Test
input = "2,you are my love. there is no spoon,you are spoon." # (1+1)/(1+8) * (0+1)/(1+8)
print("Result is {} and correct answer is {}".format(train_andPred(input), round(2.0/9*1.0/9,3)))

input = "3,you are my love. there is no spoon,you are spoon." # (0+1)/(1+8)
print("Result is {} and correct answer is {}".format(train_andPred(input), round(1.0/9,3)))

input = "4,you are my love. there is no spoon,you are spoon." # 0
print("Result is {} and correct answer is {}".format(train_andPred(input), round(0,3)))

Result is 0.025 and correct answer is 0.025
Result is 0.111 and correct answer is 0.111
Result is 0.0 and correct answer is 0.0


In [10]:
# Clean passage
import sys

# I copied the code for better debugging and checking built-in function
# on my ipython notebook IDE.


def clean(string):
    """
    Return cleaned string according to instruction
    Also returns the length of original string for comparison
    """
    n = len(string)
    string = string.lower() # to lower case
    string.strip() # remove leading and trailing whitespace 
    string = " ".join(string.split()) # remove the contiguous spaces
    
    # remove the non-alphanumeric characters
    new_string = []
    for char in string:
        if char.isalnum() or char == " ":
            new_string.append(char)
    
    new_string = "".join(new_string)
    
    return (new_string, n)

# Put everything together
for line in sys.stdin:
    passages = line.split("|")
    
    
    # Get cleaned passages
    cleaned = [clean(passage) for passage in passages]
    #print(cleaned)
    selected_i = [] # to store the index of selected clean passages 
    
    # Loop through each passage and see if it should be retained
    for i in range(len(cleaned)):
        select = True
        for j in range(len(cleaned)):
            if select == False: # no need to compare to rests
                break
            if i == j: # don't compare to itself
                continue
            item_i = cleaned[i]
            item_j = cleaned[j]
            if item_i[0] in item_j[0]: # if i is a sub-passage or equal
                if len(item_i[0]) != len(item_j[0]): # i is a sub-passage
                    select = False
                else: # i and j are equal after cleaning
                    if item_i[1] > item_j[1]: # if i's origin is longer
                        select = False
                    else: # i's origin is shorter or same
                        if item_i[1] == item_j[1] and i > j: # i is same but later
                            select = False
        if select: # i isn't lost in any j, safe to add i
            selected_i.append(i) 
    
    # Sort selected_i
    selected_i.sort()
    #print(selected_i)
    
    # Build the output with original passage
    res = [passages[i] for i in selected_i]
    
    print("|".join(res))

In [9]:
# Write a program that, given a parse tree of either a full or a partial sentence, \
# reconstructs the string. There should be no leading or trailing white spaces in the output string.

# I use a stack

In [7]:
# last 4 of a string
def string_last4(string):
    """
    Output alternated string, last 4 digits remained but others are #
    """
    s = list(string)
    if len(s) < 4:
        return "#"*len(s)
    for i in range(-4, -len(s)-1, -1):
        s[i] = "#"
    
    return "".join(s)

# output odd num
def find_odd_hash(array):
    from collections import defaultdict
    """
    Find the number in array that appears odd times
    """
    hash = defaultdict(int)
    for num in array:
        hash[num] += 1
    
    # linear scan to find numbers that appears odd times
    res = []
    for num, count in hash.items():
        if count%2 != 0:
            res.append(num)
    return res

def find_odd_bit(array):
    """
    Find the number in array that appears odd times
    """
    res = 0
    for num in array:
        res = res ^ num
    return res


In [9]:
# Test
l = [2,3,5,4,5,2,4,3,5,2,4,4,2]
print(find_odd_hash(l))
print(find_odd_bit(l))

[5]
5
